In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import os

In [ ]:
# Parameters and Settings

# File path for the raw stocks daily data
current_directory = os.getcwd()
raw_data_file = os.path.join(current_directory, 'Data', 'raw_data.csv')

# File path for the daily riskfree rate data
riskfree_rate_data_file = os.path.join(current_directory, 'Data', 'riskfree_rate_data.csv')

# Estimation (in sample) period dates
in_sample_start_date = pd.to_datetime("2000-01-01")
in_sample_end_date = pd.to_datetime("2015-12-31")

# Out-of-sample period dates
out_sample_start_date = pd.to_datetime("2016-01-01")
out_sample_end_date = pd.to_datetime("2024-12-31")

# File path to save information on top 50 stocks
top50_stocks_info_path = os.path.join(current_directory, 'Data', 'top50_stocks_info.csv')

# File path to save the cleaned and filtered data file
clean_filtered_data_path = os.path.join(current_directory, 'Data', 'clean_filtered_data.csv')

# Define necessary columns from raw stocks data
neccessary_columns = ["PERMNO", "SecurityNm", "Ticker", "SICCD", "Date", "DlyPrc", "DlyRet", "DlyVol", "ShrOut"]

# Define Energy industry SIC code ranges
energy_sic_ranges = [(1300, 1399), (4900, 4999)]

### Raw Data Information

**Raw Stocks Data:**
- Stock Daily Security Data from CRSP Stock - Version 2 (CIZ) 
- Columns: "PERMNO" (unique permanent identification numbers assigned by CRSP to all companies listed in CRSP dataset), "SecurityNm" (Security Name), "Ticker", "SICCD" (SIC Code), "DateDlyCalDt" (Date), "DlyPrc" (Daily Price), "DlyRet" (Daily Return), "DlyVol" (Daily Volume), "ShrOut" (Shares Outstanding)
- SIC Code Range: 1300-1399 (Oil and Gas Extraction), 4900-4999 (Utilities - Energy)
- Date Range: 2000-01-01 to 2024-12-31

**Risk Free Rate Data:**
- 4-week Riskfree Series from CRSP Treasuries 
- Columns: "KYTREASNOX", "TIDXFAM", "TTERMTYPE", "TTERMLBL", "CALDT", "TDYLD"

### Step 1: Load Data

In [ ]:
# Load the raw stocks daily data and daily riskfree rate data files into a pandas DataFrames
df_stocks = pd.read_csv(raw_data_file)
df_rf = pd.read_csv(riskfree_rate_data_file)

# Ensure the date columns are in datetime format
df_stocks["DlyCalDt"] = pd.to_datetime(df_stocks["DlyCalDt"])
df_stocks.rename(columns={"DlyCalDt": "Date"}, inplace=True)

df_rf["CALDT"] = pd.to_datetime(df_rf["CALDT"])
df_rf.rename(columns={"CALDT": "Date"}, inplace=True)

# Remove unnecessary columns
df_stocks = df_stocks[neccessary_columns]

print(df_stocks.head(5))
print(df_rf.head(5))

In [ ]:
# Remove rows with missing daily returns
df_stocks.dropna(subset="DlyRet", axis=0, inplace=True, ignore_index=True)

# Calculate daily excess returns using daily risk free rate
df = df_stocks.merge(df_rf[["Date", "TDYLD"]], how="left", on="Date")
df["TDYLD"] = df["TDYLD"].ffill()

df["excess_return"] = df["DlyRet"] - df["TDYLD"]

In [ ]:
# Check number of unique stocks
num_stocks = df['PERMNO'].nunique()
print(f"Number of unique stocks: {num_stocks}")

### Step 2: Clean and Filter Data

In [ ]:
# Remove rows with duplicate data
df.drop_duplicates(subset=['PERMNO', 'Date', "excess_return"], keep='first', inplace=True, ignore_index=True)

df.info(show_counts=True)

In [ ]:
# Remove stocks with incomplete data (all stocks should be traded for all trading days)

# Build the Common Trading Calendar
common_trading_days = np.sort(df["Date"].unique())

expected_days = len(common_trading_days)
print(f"Number of trading days: {expected_days}")

# Group the data by stock (PERMNO) and count distinct trading days
stock_counts = df.groupby("PERMNO")["Date"].nunique().reset_index()

# Identify stocks that have complete data
valid_stocks = stock_counts[stock_counts["Date"] == expected_days]["PERMNO"].tolist()
print(f"Number of stocks with complete data for all trading days: {len(valid_stocks)}")

# Report stocks removed
removed_stocks = stock_counts[stock_counts["Date"] < expected_days]["PERMNO"].tolist()
print(f"Number of stocks removed due to incomplete data: {len(removed_stocks)}")

# Filter the main DataFrame to keep only the stocks with complete data
df_clean = df[df["PERMNO"].isin(valid_stocks)].copy()
print(df_clean.head())
print(df_clean.shape[0])

In [ ]:
# Select the top 50 stocks by average market cap over the estimation period

# Split data into in-sample and out-of-sample dataframes
df_in_sample = df_clean[(df_clean["Date"] >= in_sample_start_date) & (df_clean["Date"] <= in_sample_end_date)].copy()

# Compute average market cap per stock over the estimation period
df_in_sample["market_cap"] = df_in_sample["DlyPrc"].abs() * df_in_sample["ShrOut"]
avg_market_cap = df_in_sample.groupby("PERMNO")["market_cap"].mean().reset_index().rename(columns={"market_cap": "avg_market_cap"})

stock_info = df_in_sample.groupby("PERMNO").agg({"Ticker": "last", "SecurityNm": "last"}).reset_index()   # Using "last" to get newest/latest ticker and company name if a company undergoes name change, etc.
avg_market_cap = avg_market_cap.merge(stock_info, on="PERMNO", how="left")

# Select the top 50 stocks
top50_df = avg_market_cap.sort_values(by="avg_market_cap", ascending=False).head(50)
top50_permno = top50_df["PERMNO"].tolist()

print("Top 50 stocks:")
top50_df

In [ ]:
# Save information about top 50 stocks by average market cap
top50_df.to_csv(top50_stocks_info_path)

In [ ]:
# Filter data to include only top 50 stocks by average market cap over the estimation period
df_filtered = df_clean[df_clean["PERMNO"].isin(top50_permno)].copy().reset_index()
df_filtered.drop(["SICCD", "DlyPrc", "DlyRet", "DlyVol", "ShrOut", "TDYLD"], axis=1, inplace=True)
print("Cleaned and filtered data:")
print(df_filtered.info())

### Step 3: Save Cleaned and Filtered Data

In [ ]:
# Save clean and filtered 
df_filtered = df_filtered.sort_values(['PERMNO', 'Date'])
df_filtered.to_csv(clean_filtered_data_path)